### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
from importlib import reload

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [2]:
reload(dl)
reload(sf)
reload(fit)
reload(bs)

<module 'fitter.bootstrapper' from '../fitter/bootstrapper.py'>

### Specify fit parameters

In [3]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True, # alphaS
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'xpt', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'chained' : False,
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
}

In [4]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], F2=p_dict['F2'], include_FV=(p_dict['order']['vol']>0), 
                                 include_alphaS=p_dict['order']['include_log'], include_logSq=p_dict['order']['include_log2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(
    fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'], 
    include_su2_isospin_corrrection=p_dict['include_su2_isospin_corrrection'],
    fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct']
)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    pass #print(bootstrapper)
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.

Total time (s):  0.6530518531799316 



In [5]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


bootstrapper.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

eps2_pi 0.013576441944866667
eps2_k 0.18289664858239507
eps2_eta 0.2393367174615712
1 + nlo 1.2098507616726104
ct -0.08370923117181872
extra ct 0.009271430036091247
mu fix -0.0605200744906015
FF 0.012421152702167185
logSq 0.028598430896321657
log 0.035831497394029015
nnlo -0.05810679463381119
p {'L_1': 0.000372, 'L_2': 0.000493, 'L_3': -0.00307, 'L_4': 8.9e-05, 'L_5': 0.000377, 'L_6': 1.1e-05, 'L_7': -0.00034, 'L_8': 0.000294, 'A_k': -3.0, 'A_p': 4.0, 'mpi': 135.0, 'mk': 495.5, 'Fpi': 92.2, 'FK': 110.5, 'lam2_chi': 1342398.8460313035}


1.1517439670387992

In [6]:
eps2_pi = fit_data['mpi']**2 / fit_data['lam2_chi']
eps2_k = fit_data['mk']**2 / fit_data['lam2_chi']

sf.fcn_Cr_j(1, eps2_pi, eps2_k, fit_parameters)

-0.002524380496727892

In [10]:
sf.fcn_Cr_j(2, eps2_pi, eps2_k, fit_parameters)

-0.034624468492324194

In [11]:
sf.fcn_Cr_j(3, eps2_pi, eps2_k, fit_parameters)

0.023668424759729596